<a href="https://colab.research.google.com/github/brownnyi/Project_FakeNews-Detector/blob/main/%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%B6%84%EC%84%9D_%EB%B0%8F_%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
!pip install contractions
import contractions
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer, WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, precision_score, \
                             recall_score, f1_score, roc_curve, \
                             accuracy_score, roc_auc_score
from sklearn.model_selection import ParameterGrid, GridSearchCV


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers as L
from tensorflow.keras.layers import  Embedding
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Dense, Dropout, LSTM, Embedding, GRU, SimpleRNN
from tensorflow.keras.models import Sequential
from tensorflow.python.keras.callbacks import EarlyStopping

from __future__ import absolute_import, division, print_function, unicode_literals

VOCAB_SIZE = 5000

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 8.2 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')
submit = pd.read_csv('submit.csv')

In [ ]:
test

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...
...,...,...,...,...
5195,25995,The Bangladeshi Traffic Jam That Never Ends - ...,Jody Rosen,Of all the dysfunctions that plague the world’...
5196,25996,John Kasich Signs One Abortion Bill in Ohio bu...,Sheryl Gay Stolberg,WASHINGTON — Gov. John Kasich of Ohio on Tu...
5197,25997,"California Today: What, Exactly, Is in Your Su...",Mike McPhate,Good morning. (Want to get California Today by...
5198,25998,300 US Marines To Be Deployed To Russian Borde...,NaN,« Previous - Next » 300 US Marines To Be Deplo...


In [ ]:
train

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [ ]:
submit

,id,label
0,20800,0
1,20801,1
2,20802,0
3,20803,1
4,20804,1
...,...,...
5195,25995,0
5196,25996,1
5197,25997,0
5198,25998,1


In [ ]:
def data_clean(dt):
    dt = dt.fillna('')
    dt['total'] = dt['title'] + "" + dt['author']
    return dt

def text_cleaning(df):
    ps = PorterStemmer()
    corpus = []
    for i in range(len(df)):
        m = re.sub("[^a-zA-Z]","",df['total'][i])
        m = m.lower()
        m = m.split()
        m = [ps.stem(word) for word in m if not word in stopwords.words('english')]
        clean_text = "".join(m)
        corpus.append(clean_text)
    return corpus

def onehot(corpus, VOCAB_SIZE = 5000):
    return [one_hot(words, VOCAB_SIZE) for words in corpus]

def padding(onehot_text):
    return np.array(pad_sequences(onehot_text, padding = 'pre', maxlen = 25))

def get_label(df):
    return np.array(df['label'])

def make_clean(df):
    corpus = data_clean(df)
    corpus = text_cleaning(corpus)
    corpus = onehot(corpus)
    corpus = padding(corpus)
    return corpus

def get_data():
    test = pd.read_csv('test.csv')
    train = pd.read_csv('train.csv')
    submit = pd.read_csv('submit.csv')
    x = np.array(list(make_clean(train)) + list(make_clean(test)))
    y = np.array(list(get_label(train)) + list(get_label(submit)))
    return train_test_split(x, y, test_size = 0.1, random_state = 1234)

In [ ]:
train_x, test_x, train_y, test_y = get_data()


In [ ]:
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size = 0.1, random_state = 1234)

In [ ]:
train_x.shape, train_y.shape, val_x.shape, val_y.shape, test_x.shape, test_y.shape

((21060, 25), (21060,), (2340, 25), (2340,), (2600, 25), (2600,))

In [ ]:
lstm = Sequential()
lstm.add(Embedding(30000, 30))
lstm.add(keras.layers.Dropout(0.3))
lstm.add(LSTM(100))
lstm.add(keras.layers.Dropout(0.3))
lstm.add(Dense(64, activation = 'relu'))
lstm.add(keras.layers.Dropout(0.3))
lstm.add(Dense(1, activation = 'sigmoid'))
lstm.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['acc'])
history = lstm.fit(train_x, train_y, epochs = 100, validation_data = (val_x, val_y))

Epoch 1/100
659/659 [==============================] - 35s 48ms/step - loss: 0.6933 - acc: 0.5055 - val_loss: 0.6934 - val_acc: 0.5026
Epoch 2/100
659/659 [==============================] - 35s 53ms/step - loss: 0.6868 - acc: 0.5415 - val_loss: 0.6946 - val_acc: 0.4957
Epoch 3/100
659/659 [==============================] - 31s 47ms/step - loss: 0.6373 - acc: 0.6356 - val_loss: 0.7236 - val_acc: 0.5098
Epoch 4/100
659/659 [==============================] - 32s 49ms/step - loss: 0.6098 - acc: 0.6591 - val_loss: 0.7543 - val_acc: 0.5162
Epoch 5/100
659/659 [==============================] - 29s 44ms/step - loss: 0.5964 - acc: 0.6641 - val_loss: 0.8108 - val_acc: 0.5154
Epoch 6/100
659/659 [==============================] - 30s 45ms/step - loss: 0.5880 - acc: 0.6685 - val_loss: 0.8054 - val_acc: 0.5154
Epoch 7/100
659/659 [==============================] - 29s 44ms/step - loss: 0.5824 - acc: 0.6716 - val_loss: 0.8338 - val_acc: 0.5150
Epoch 8/100
659/659 [==============================] - 

In [ ]:
def show_metrics(y_pred, y_actual) :
  print('Accuracy  :',  accuracy_score(y_pred, y_actual))
  print('Precision :', precision_score(y_pred, y_actual))
  print('Recall    :',    recall_score(y_pred, y_actual))
  print('F1-score  :',        f1_score(y_pred, y_actual))
  print('Auc       :',   roc_auc_score(y_pred, y_actual))

In [ ]:
ls ./drive/MyDrive/멀티캠퍼스_5회차/data/*

 ./drive/MyDrive/멀티캠퍼스_5회차/data/1C0C8FA2-1FF9-45B2-8552-3341DC577E7B_images_with_product_ids.zip
 ./drive/MyDrive/멀티캠퍼스_5회차/data/atlas-higgs-challenge-2014-v2.csv.gz
 ./drive/MyDrive/멀티캠퍼스_5회차/data/bike_rentals.csv
 ./drive/MyDrive/멀티캠퍼스_5회차/data/bikes_cleaned.csv
 ./drive/MyDrive/멀티캠퍼스_5회차/data/bike_train.csv
 ./drive/MyDrive/멀티캠퍼스_5회차/data/boston_house.csv
 ./drive/MyDrive/멀티캠퍼스_5회차/data/California_Houses.csv
 ./drive/MyDrive/멀티캠퍼스_5회차/data/citrus.csv
 ./drive/MyDrive/멀티캠퍼스_5회차/data/creditcard.csv
 ./drive/MyDrive/멀티캠퍼스_5회차/data/credit_card.xls
 ./drive/MyDrive/멀티캠퍼스_5회차/data/data_incomes.csv
 ./drive/MyDrive/멀티캠퍼스_5회차/data/diabetes.csv
 ./drive/MyDrive/멀티캠퍼스_5회차/data/exoplanets.csv
 ./drive/MyDrive/멀티캠퍼스_5회차/data/exoplanets.csv.zip
 ./drive/MyDrive/멀티캠퍼스_5회차/data/features.txt
 ./drive/MyDrive/멀티캠퍼스_5회차/data/heart_disease.csv
 ./drive/MyDrive/멀티캠퍼스_5회차/data/hpr_data.csv
 ./drive/MyDrive/멀티캠퍼스_5회차/data/image_SVD_data.jpg
 ./drive/MyDrive/멀티캠퍼스_5회차/data/labeledTrainData.tsv
 ./drive/My

In [ ]:
import pandas as pd

In [ ]:
df_train = pd.read_csv('./drive/MyDrive/멀티캠퍼스_5회차/data/낚시성 데이터/Traing_merged_data-001.csv')

<ipython-input-4-d0fb440d4954>:1: DtypeWarning: Columns (14,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv('./drive/MyDrive/멀티캠퍼스_5회차/data/낚시성 데이터/Traing_merged_data-001.csv')


In [ ]:
df_val = pd.read_csv('./drive/MyDrive/멀티캠퍼스_5회차/data/낚시성 데이터/Validation_merged_data.csv')

<ipython-input-18-996d6ce59869>:1: DtypeWarning: Columns (14,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df_val = pd.read_csv('./drive/MyDrive/멀티캠퍼스_5회차/data/낚시성 데이터/Validation_merged_data.csv')


In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1015846 entries, 0 to 1015845
Data columns (total 18 columns):
 #   Column                               Non-Null Count    Dtype  
---  ------                               --------------    -----  
 0   sourceDataInfo.newsID                1015846 non-null  object 
 1   sourceDataInfo.newsCategory          1015846 non-null  object 
 2   sourceDataInfo.newsSubcategory       1013104 non-null  object 
 3   sourceDataInfo.newsTitle             1015846 non-null  object 
 4   sourceDataInfo.newsSubTitle          336849 non-null   object 
 5   sourceDataInfo.newsContent           1015846 non-null  object 
 6   sourceDataInfo.partNum               1015846 non-null  object 
 7   sourceDataInfo.useType               1015846 non-null  int64  
 8   sourceDataInfo.processType           1015846 non-null  object 
 9   sourceDataInfo.processPattern        1015846 non-null  int64  
 10  sourceDataInfo.processLevel          1015846 non-null  object 
 11

In [ ]:
df_val

In [ ]:
df_train.iloc[:1,:][['sourceDataInfo.newsContent']]

,sourceDataInfo.newsContent
0,"#. 서울의 한 카페, 아르바이트생이 마감 청소 중 춤을 추고 있다.\n춤 솜씨가 예사롭지 않다.\n아무도 없는 시간이었지만 조용히 문이 열리고 한 손님이 그 광경을 지켜본다.\n아르바이트생이 뒤를 돌아 보자 서로 눈이 마주치고 멋쩍게 웃는다.\n이 광경은 에스원 CCTV로 촬영됐다.\n보안 기업 에스원이 자사 CCTV를 홍보하는 영상 콘텐츠가 전 세계에서 인기몰이를 하고 있다.\n한 달 만에 유튜브, 틱톡 등 글로벌 소셜네트워크서비스(SNS)에서 누적 조회 수 3500만을 돌파해 해외 유명 방송국의 취재 요청까지 들어오고 있다.\n30일 에스원에 따르면 유튜브 공식 채널인 ‘어서와 에스원’ 구독자 수는 전년 대비 380%, 조회 수는 656% 증가했다.\n특히 유튜버들과 협업한 영상 콘텐츠들은 평균 60만회가 넘는 조회수를 기록하는 등 채널의 인기 상승 동력이 되고 있다.\n최근 화제가 된 CCTV 영상도 걸그룹 ITZY 노래에 맞춰 추는 춤 덕분에 ‘ITZY 빌런’으로 유명해진 유튜버 ‘효크포크’와 협업해 만들었다. ‘보안업은 경직되고 딱딱하다’는 고정관념을 깨기 위해 제작했다는 게 에스원의 설명이다.\n최근 SNS를 마케팅에 활용하는 기업의 숫자는 증가하고 있지만 성공사례는 손에 꼽는다는 점에서 에스원의 ‘대박’은 눈에 띈다.\n국내 기업 중 SNS에서 1000만회 넘는 조회수를 기록한 영상은 삼성전자의 ‘비스포크’ 등과 같이 TV광고를 SNS에 함께 공개한 것이거나 큰 제작비를 투입한 게 대부분이다.\n에스원은 입소문만으로 3500만뷰를 달성했다.\n에스원은 올해부터 MZ세대(밀레니얼+Z세대)를 공략하기 위해 SNS에 재밌고 유쾌한 콘텐츠를 잇달아 선보이며 꾸준히 팬덤을 확보해왔다고 설명했다.\n에스원 관계자는 \""멀게만 느껴졌던 보안서비스가 우리 생활 곳곳에서 쉽게 만나볼 수 있는 서비스임을 홍보해 보안업의 대중화를 선도하겠다\""며 \""에스원은 기업 SNS채널의 한계를 넘어 대중들이 재밌게 즐길 수 있는 콘텐츠를 계속해서 선보이겠다\""고 말했다."


In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
df_val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126948 entries, 0 to 126947
Data columns (total 18 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   sourceDataInfo.newsID                126948 non-null  object 
 1   sourceDataInfo.newsCategory          126948 non-null  object 
 2   sourceDataInfo.newsSubcategory       126528 non-null  object 
 3   sourceDataInfo.newsTitle             126948 non-null  object 
 4   sourceDataInfo.newsSubTitle          33751 non-null   object 
 5   sourceDataInfo.newsContent           126948 non-null  object 
 6   sourceDataInfo.partNum               126948 non-null  object 
 7   sourceDataInfo.useType               126948 non-null  int64  
 8   sourceDataInfo.processType           126948 non-null  object 
 9   sourceDataInfo.processPattern        126948 non-null  int64  
 10  sourceDataInfo.processLevel          126948 non-null  object 
 11  sourceDataInf

In [ ]:
73798 + 36422

110220